[comment]: <> "LTeX: language=fr"
# <center> Formation IXXI - Grands modèles de langage </center>
## <center> Jean-Philippe Magué </center>

<div align="center">
	<img src="img/logo_ixxi.png">
</div>



Cette formation s'appuie sur deux notebooks. Ce premier propose une vision de "haut niveau" du fonctionnement du modèle [GPT2](https://huggingface.co/openai-community/gpt2), c'est-à-dire en regardant les entrées et les sorties, mais sans rentrer dans le fonctionnement interne. Le second entre dans les détails internes du modèle. 

## <center> LLM : Vision de haut niveau </center>


# Initialisation

Ces notebooks utilisent la bibliothèque [transformers](https://huggingface.co/docs/transformers/en/index) de [HuggingFace](https://huggingface.co/). Cette bibliothèque permet de manipuler une grande variété de modèles, notamment ceux disponibles sur [HuggingFace](https://huggingface.co/).

In [27]:
try:
  import google.colab
  import subprocess
  import sys

  def install(package):
      subprocess.check_call([sys.executable, "-m", "pip", "install", package])
  install('accelerate')
except:
  pass

In [28]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import torch

In [29]:
torch.set_grad_enabled(False) #no training today!

In [30]:
model_name = 'gpt2'

In [31]:
# On charge un modèle pré-entrainé à partir de son nom
tokenizer = GPT2TokenizerFast.from_pretrained(model_name) 
gpt2 = GPT2LMHeadModel.from_pretrained(model_name,device_map="auto",pad_token_id=tokenizer.eos_token_id)# device_map="auto" permet d'utiliser le GPU si il est disponible
device = gpt2.device 
device #indique si le GPU est utilisé (cuda ou mps) ou non (cpu)

device(type='mps', index=0)

# Premières générations de texte

[comment]: <> "LTeX: language=fr"
## Tokenisation
Les modèles de langue ne traitent pas directement des chaînes de caractères. Celles-ci sont **tokenisées** et ce sont les tokens qui consituent les unités traitées par ces modèles. La liste de ces tokens est finie (il y en a 50257 pour GPT2) et ils sont définis avant même l'entraîment du modèle. 

In [32]:
print(f'il y a {gpt2.config.vocab_size} tokens dans le vocabulaire')

il y a 50257 tokens dans le vocabulaire


In [47]:
prompt = 'Artificial intelligence'
prompt = 'ArtificCial intElliGence'

In [48]:
input = tokenizer(prompt, return_tensors="pt").to(device) 
# return_tensors="pt" indique que l'on veut des tenseurs pytorch
# to(device) permet de mettre les données sur le GPU si il est disponible
input

{'input_ids': tensor([[ 8001,   811,    34,   498,   493,    36, 15516,    38,   594]],
       device='mps:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='mps:0')}

In [49]:
print(f'{"|token|":8} -> id\n')
for token in input['input_ids'][0]:
    print(f'{"|"+tokenizer.decode(token)+"|":8} -> {token}')

|token|  -> id

|Art|    -> 8001
|ific|   -> 811
|C|      -> 34
|ial|    -> 498
| int|   -> 493
|E|      -> 36
|lli|    -> 15516
|G|      -> 38
|ence|   -> 594


[comment]: <> "LTeX: language=fr"
#### Un mot sur les batchs
Ces modèles sont prévus pour traiter plusieurs entrées (plusieurs prompts) en même temps. On appelle l'ensemble de ces entrées un *batch*. Dans ce notebook et le suivant, nous ne traiterons toujours qu'une seule entrée à la fois (nos batchs ont une taille de 1).

Dans la cellule ci-dessous, pour accéder aux tokens de l'entrée, on utilise :

```python 
input['input_ids'][0]
```

L'indice `0`indique que l'on s'intéresse à la première (et unique) des entrées du batch. Cet indice apparaîtra à de nombreuses reprises dans ces notebooks.

Par ailleurs, il faut que les entrées d'un batch aient toutes le même nombre de tokens. Quand ce n'est pas le cas, le modèle complète les plus courtes avec un token spécial, nommé *padding token*. D'où le paramètre `pad_token_id=tokenizer.eos_token_id`lors de la création du modèle.

## Génération

In [14]:
output = gpt2.generate(input_ids=input['input_ids'], attention_mask=input['attention_mask'], max_new_tokens=1)#le attention_mask permet de dire au de ne pas prêter attention aux tokens de padding

In [15]:
output

tensor([[8001, 9542, 4430,  318]], device='mps:0')

[comment]: <> "LTeX: language=fr"
Le modèle renvoie également une liste (d'identifiants) de tokens : ceux donnés en entrée plus celui généré. Le tokenizer permet de les décoder.

In [16]:
print(f'{"|token|":8} -> id\n')
for token in output[0]:
    print(f'{"|"+tokenizer.decode(token)+"|":8} -> {token}')

|token|  -> id

|Art|    -> 8001
|ificial| -> 9542
| intelligence| -> 4430
| is|    -> 318


Si l'on souhaite générer plus de 1 token, il suffit de répéter l'opération:

In [17]:
output2 = gpt2.generate(input_ids=output, attention_mask=torch.ones(output.shape).to(device), max_new_tokens=1)
tokenizer.decode(output2[0])

'Artificial intelligence is a'

In [18]:
output3 = gpt2.generate(input_ids=output2, attention_mask=torch.ones(output2.shape).to(device), max_new_tokens=1)
tokenizer.decode(output3[0])

'Artificial intelligence is a new'

In [19]:
output4 = gpt2.generate(input_ids=output3, attention_mask=torch.ones(output3.shape).to(device), max_new_tokens=1)
tokenizer.decode(output4[0])

'Artificial intelligence is a new field'

Ou alors, plus simplement : 

In [20]:
output10 = gpt2.generate(input_ids=input['input_ids'], attention_mask=input['attention_mask'], max_new_tokens=6)
tokenizer.decode(output10[0])

'Artificial intelligence is a new field of research'

[comment]: <> "LTeX: language=fr"
# Stratégies de génération de texte
Ce que calcule le modèle est en fait un score associé à chacun des 50257 tokens à partir duquel il est possible de dériver une distribution de probabilité sur les tokens. Le choix du token qui sera généré est effectué à partir de ces scores/probabilités. On peut demander au modèle de renvoyer ces scores avec les paramètres `return_dict_in_generate=True`, `output_scores=True`

In [33]:
output = gpt2.generate(input_ids=input['input_ids'], attention_mask=input['attention_mask'], max_new_tokens=1,return_dict_in_generate=True, output_scores=True)

In [34]:
output['scores']

(tensor([[ -99.0574,  -97.3374, -104.6698,  ..., -105.8733, -106.3209,
           -99.8503]], device='mps:0'),)

[comment]: <> "LTeX: language=fr"
Ces scores (*logits*) sont transformés en probabilités via une fonction [softmax](https://stackoverflow.com/a/47763299/1731620) :

$\sigma(x_i)=\frac{e^{x_i}}{\sum{e^{x_j}}}$

In [35]:
prob = torch.nn.functional.softmax(output['scores'][0],dim=1)
prob=prob[0].cpu()
prob_of_each_token = list(zip([tokenizer.decode(i) for i in range(50257)], prob))

[comment]: <> "LTeX: language=fr"
On peut regarder quels sont les 10 tokens avec la plus haute probabilité

In [36]:
print(f'{"|token|":9} -> probability\n')
for token, p in sorted(prob_of_each_token, key=lambda x: x[1], reverse=True)[:10]:
    print(f'{"|"+token+"|":9} -> {p:.4f}')

|token|   -> probability

| is|     -> 0.1954
| (|      -> 0.0700
|,|       -> 0.0597
| has|    -> 0.0580
| and|    -> 0.0462
| will|   -> 0.0447
| can|    -> 0.0371
| could|  -> 0.0173
|.|       -> 0.0166
| may|    -> 0.0135


### Greedy search

[comment]: <> "LTeX: language=fr"
[Différentes stratégies sont possibles](https://huggingface.co/blog/how-to-generate) pour générer le prochain token. La stratégie par défaut employée ici, **greedy search**, consiste tout simplement à choisir à chaque étape le token ayant la plus grande probabilité :


<img src="https://raw.githubusercontent.com/jmague/formation_LLM_IXXI/master/img/images/images.001.png" width="600">


Elle atteint assez vite ses limites : 

In [37]:
output = gpt2.generate(input_ids=input['input_ids'], attention_mask=input['attention_mask'], max_new_tokens=50)
tokenizer.decode(output[0])

'Artificial intelligence is a new field of research that has been in the works for a while now. It is a field that has been in the works for a while now. It is a field that has been in the works for a while now. It is a'

[comment]: <> "LTeX: language=fr"
### Un peu de stochasticité
On peut être moins détermistes et profiter de la distribution de probabilité pour introduire de l'aléatoire :


In [40]:
output = gpt2.generate(input_ids=input['input_ids'], attention_mask=input['attention_mask'], max_new_tokens=50, do_sample=True)
tokenizer.decode(output[0])

'Artificial intelligence (AI) and machine learning techniques have been found in some papers to be useful in predicting people\'s views of robots and robots and robots. In the early paper on AI researchers pointed out the potential for these techniques to improve on the old "predict'

[comment]: <> "LTeX: language=fr"
### Beam search
Cette stratégie n'est pas la plus efficace. La séquence de n tokens la plus probable peut ne pas commencer par le token le plus probable :

<img src="https://raw.githubusercontent.com/jmague/formation_LLM_IXXI/master/img/images/images.002.png" width="600">

Le principe du **Beam search** est de générer en parallèle plusieurs séquences de tokens, de manière à produire, *in fine*, la plus probable

In [46]:
#num_beams : nombre de sequences paralleles maintenues
output = gpt2.generate(input_ids=input['input_ids'], attention_mask=input['attention_mask'], max_new_tokens=70, num_beams = 5, do_sample=True)
tokenizer.decode(output[0])

'Artificial intelligence (AI) is becoming more and more popular in the digital age.\n\nAccording to a new report by the American Society of Artificial Intelligence (ASI), AI is becoming more and more popular in the digital age. According to a new report by the American Society of Artificial Intelligence (ASI), AI is becoming more and more popular in the digital'

In [50]:
prompt = "What is the capital of Australia?"
input = tokenizer(prompt, return_tensors="pt").to(device)
output = gpt2.generate(input_ids=input['input_ids'], attention_mask=input['attention_mask'], max_new_tokens=50, num_beams = 5, do_sample=True)
print(tokenizer.decode(output[0]))

What is the capital of Australia?

The capital of Australia is the state capital of Australia. The capital of Australia is the state capital of Australia. The capital of Australia is the state capital of Australia. The capital of Australia is the state capital of Australia. The capital of Australia
